# Classification of Reviews


The following notebooks details how different types 

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
# change directory
os.chdir("/Users/andreamock/Documents/")

In [4]:
negativeDf = pd.read_csv('negativeReviews2000.csv', index_col=0)
positiveDf = 

In [8]:
posDf = pd.read_excel('Positive_reviews.xlsx')
posDf.head()

,username,user_url,num_stars,review_text,lang,review_date,type,scraped_from,Relevant,Sentiment,Topic,Key words,Objects,Names
0,Nate Neel,https://www.google.com/maps/contrib/1121030547...,5,"Waterfront to fish or just relax, great place ...",en,2020-10-20 22:04:09,Park,Google Maps,yes,positive,walkability,"picnic,hangout,relax",NaN,NaN
1,Johnny Evangelista,https://www.google.com/maps/contrib/1081264861...,5,"Tennis courts, soccer field, 1 basketball net....",en,2021-04-22 11:54:53,Park,Google Maps,yes,positive,sports facilities,"tennis courts,soccer field","tennis courts,soccer field , basketball net",NaN
2,Desmond Yu,https://www.google.com/maps/contrib/1174595133...,5,"Kids can play in the playground, enjoying outd...",en,2021-04-22 11:21:25,Park,Google Maps,yes,positive,sports facilities,"playground,game,soccer","park,playground",NaN
3,Desmond Yu,https://www.google.com/maps/contrib/1174595133...,5,Couples can take a walk or find a bench and ch...,en,2021-04-22 11:21:25,Park,Google Maps,yes,positive,walkability,"walk,bench",bench,NaN
4,irene karpontinis,https://www.google.com/maps/contrib/1077994536...,5,I took a most enjoyable stroll admiring the lu...,en,2018-06-22 11:21:25,Park,Google Maps,yes,positive,walkability,stroll,"athletic fields,park,cycling paths,",NaN


In [13]:
posDf = posDf[posDf['Sentiment'] == 'positive'] # filter out only positive reviews

In [15]:
posDf.columns

Index(['username', 'user_url', 'num_stars', 'review_text', 'lang',
       'review_date', 'type', 'scraped_from', 'Relevant', 'Sentiment', 'Topic',
       'Key words', 'Objects', 'Names'],
      dtype='object')

In [17]:
negativeDf.columns

Index(['location', 'review_date', 'lang', 'relevant', 'sentiment', 'key words',
       'objects', 'names', 'topic', 'review_text', 'scraped_from', 'num_stars',
       'type'],
      dtype='object')

In [19]:
posDf.columns = posDf.columns.str.lower()

In [23]:
allReviewsDf = pd.concat([negativeDf,posDf], ignore_index=True)
allReviewsDf.head()

,location,review_date,lang,relevant,sentiment,key words,objects,names,topic,review_text,scraped_from,num_stars,type,username,user_url
0,"6826 The Avenue Côte Saint-Luc, Quebec",2014-05-22 17:10:20,en,yes,negative,"sidewalk, asphalted, hazard, car",sidewalk,NaN,roads and sidewalks,The sidewalk/curb that extends from bike path ...,SeeClickFix,NaN,NaN,NaN,NaN
1,"7500 Chemin Mackle Côte Saint-Luc, QC H4W 1A6,...",2014-06-03 23:42:45,en,yes,negative,"chairs, railing, fall","chairs, railing",NaN,municipal buildings,Chairs too close to railing. Kids could climb ...,SeeClickFix,NaN,NaN,NaN,NaN
2,"5752 Av :Lockwood Cote Saint Luc , Quebec",2015-03-06 15:21:10,en,yes,negative,"garbage, bin, color, trash","bin, city",NaN,cleanliness,THE COLOR OF A BIN SHOULD NOT MATTER. THURSDAY...,SeeClickFix,NaN,NaN,NaN,NaN
3,5581 Rosedale H4V 2J3,2017-08-02 13:26:54,en,yes,negative,"people, tripped, injured, broken, sidewalks",sidewalks,NaN,roads and sidewalks,Two people have already tripped and injured th...,SeeClickFix,NaN,NaN,NaN,NaN
4,"5550-5556 Avenue Lyndale Cote-Saint-Luc, QC H4...",2013-11-14 13:08:48,en,yes,negative,street sign,street sign,Lyndale,traffic lights and signs,"New street sign says ""lyndal"" instead of ""lynd...",SeeClickFix,NaN,NaN,NaN,NaN


In [27]:
# only keep relevant comments for processing
allReviewsDf = allReviewsDf[['sentiment', 'key words', 'objects', 'names', 'topic', 'review_text']]

In [30]:
# create column with label 1 for positive, 0 for negative
allReviewsDf['label'] = allReviewsDf['sentiment'].replace({'negative': 0, 'positive': 1})

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Cleaning data

In [32]:
import re
def clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df

In [36]:
data_clean = allReviewsDf.copy()

In [37]:
data_clean= clean_text(data_clean, 'review_text', 'text_clean')
data_clean.head()

,sentiment,key words,objects,names,topic,review_text,label,text_clean
0,negative,"sidewalk, asphalted, hazard, car",sidewalk,NaN,roads and sidewalks,The sidewalk/curb that extends from bike path ...,0,the sidewalkcurb that extends from bike path t...
1,negative,"chairs, railing, fall","chairs, railing",NaN,municipal buildings,Chairs too close to railing. Kids could climb ...,0,chairs too close to railing kids could climb u...
2,negative,"garbage, bin, color, trash","bin, city",NaN,cleanliness,THE COLOR OF A BIN SHOULD NOT MATTER. THURSDAY...,0,the color of a bin should not matter thursday ...
3,negative,"people, tripped, injured, broken, sidewalks",sidewalks,NaN,roads and sidewalks,Two people have already tripped and injured th...,0,two people have already tripped and injured th...
4,negative,street sign,street sign,Lyndale,traffic lights and signs,"New street sign says ""lyndal"" instead of ""lynd...",0,new street sign says lyndal instead of lyndale


## Removing stopwords

In [38]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andreamock/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
stop = stopwords.words('english')
data_clean['text_clean'] = data_clean['text_clean'].apply(lambda x: ' '.join(
    [word for word in x.split() if word not in (stop)]))
data_clean.head()

,sentiment,key words,objects,names,topic,review_text,label,text_clean
0,negative,"sidewalk, asphalted, hazard, car",sidewalk,NaN,roads and sidewalks,The sidewalk/curb that extends from bike path ...,0,sidewalkcurb extends bike path road aligned dr...
1,negative,"chairs, railing, fall","chairs, railing",NaN,municipal buildings,Chairs too close to railing. Kids could climb ...,0,chairs close railing kids could climb fall dea...
2,negative,"garbage, bin, color, trash","bin, city",NaN,cleanliness,THE COLOR OF A BIN SHOULD NOT MATTER. THURSDAY...,0,color bin matter thursday garbage day garbage ...
3,negative,"people, tripped, injured, broken, sidewalks",sidewalks,NaN,roads and sidewalks,Two people have already tripped and injured th...,0,two people already tripped injured night broke...
4,negative,street sign,street sign,Lyndale,traffic lights and signs,"New street sign says ""lyndal"" instead of ""lynd...",0,new street sign says lyndal instead lyndale


### Perform word stemming

In [40]:
from nltk.stem import SnowballStemmer 
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andreamock/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
data_clean['text_tokens'] = data_clean['text_clean'].apply(lambda x: word_tokenize(x))
data_clean.head()

,sentiment,key words,objects,names,topic,review_text,label,text_clean,text_tokens
0,negative,"sidewalk, asphalted, hazard, car",sidewalk,NaN,roads and sidewalks,The sidewalk/curb that extends from bike path ...,0,sidewalkcurb extends bike path road aligned dr...,"[sidewalkcurb, extends, bike, path, road, alig..."
1,negative,"chairs, railing, fall","chairs, railing",NaN,municipal buildings,Chairs too close to railing. Kids could climb ...,0,chairs close railing kids could climb fall dea...,"[chairs, close, railing, kids, could, climb, f..."
2,negative,"garbage, bin, color, trash","bin, city",NaN,cleanliness,THE COLOR OF A BIN SHOULD NOT MATTER. THURSDAY...,0,color bin matter thursday garbage day garbage ...,"[color, bin, matter, thursday, garbage, day, g..."
3,negative,"people, tripped, injured, broken, sidewalks",sidewalks,NaN,roads and sidewalks,Two people have already tripped and injured th...,0,two people already tripped injured night broke...,"[two, people, already, tripped, injured, night..."
4,negative,street sign,street sign,Lyndale,traffic lights and signs,"New street sign says ""lyndal"" instead of ""lynd...",0,new street sign says lyndal instead lyndale,"[new, street, sign, says, lyndal, instead, lyn..."


In [42]:
def word_stemmer(text,lang):
    stem_text = [SnowballStemmer(lang).stem(i) for i in text]
    return stem_text

In [43]:
data_clean['text_tokens_stem'] = data_clean['text_tokens'].apply(lambda x: word_stemmer(x, 'english'))
data_clean.head()

,sentiment,key words,objects,names,topic,review_text,label,text_clean,text_tokens,text_tokens_stem
0,negative,"sidewalk, asphalted, hazard, car",sidewalk,NaN,roads and sidewalks,The sidewalk/curb that extends from bike path ...,0,sidewalkcurb extends bike path road aligned dr...,"[sidewalkcurb, extends, bike, path, road, alig...","[sidewalkcurb, extend, bike, path, road, align..."
1,negative,"chairs, railing, fall","chairs, railing",NaN,municipal buildings,Chairs too close to railing. Kids could climb ...,0,chairs close railing kids could climb fall dea...,"[chairs, close, railing, kids, could, climb, f...","[chair, close, rail, kid, could, climb, fall, ..."
2,negative,"garbage, bin, color, trash","bin, city",NaN,cleanliness,THE COLOR OF A BIN SHOULD NOT MATTER. THURSDAY...,0,color bin matter thursday garbage day garbage ...,"[color, bin, matter, thursday, garbage, day, g...","[color, bin, matter, thursday, garbag, day, ga..."
3,negative,"people, tripped, injured, broken, sidewalks",sidewalks,NaN,roads and sidewalks,Two people have already tripped and injured th...,0,two people already tripped injured night broke...,"[two, people, already, tripped, injured, night...","[two, peopl, alreadi, trip, injur, night, brok..."
4,negative,street sign,street sign,Lyndale,traffic lights and signs,"New street sign says ""lyndal"" instead of ""lynd...",0,new street sign says lyndal instead lyndale,"[new, street, sign, says, lyndal, instead, lyn...","[new, street, sign, say, lyndal, instead, lyndal]"


### Word lemmatization

In [44]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andreamock/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [45]:
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [46]:
data_clean['text_tokens_lemma'] = data_clean['text_tokens'].apply(lambda x: word_lemmatizer(x))
data_clean.head()

,sentiment,key words,objects,names,topic,review_text,label,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma
0,negative,"sidewalk, asphalted, hazard, car",sidewalk,NaN,roads and sidewalks,The sidewalk/curb that extends from bike path ...,0,sidewalkcurb extends bike path road aligned dr...,"[sidewalkcurb, extends, bike, path, road, alig...","[sidewalkcurb, extend, bike, path, road, align...","[sidewalkcurb, extends, bike, path, road, alig..."
1,negative,"chairs, railing, fall","chairs, railing",NaN,municipal buildings,Chairs too close to railing. Kids could climb ...,0,chairs close railing kids could climb fall dea...,"[chairs, close, railing, kids, could, climb, f...","[chair, close, rail, kid, could, climb, fall, ...","[chair, close, railing, kid, could, climb, fal..."
2,negative,"garbage, bin, color, trash","bin, city",NaN,cleanliness,THE COLOR OF A BIN SHOULD NOT MATTER. THURSDAY...,0,color bin matter thursday garbage day garbage ...,"[color, bin, matter, thursday, garbage, day, g...","[color, bin, matter, thursday, garbag, day, ga...","[color, bin, matter, thursday, garbage, day, g..."
3,negative,"people, tripped, injured, broken, sidewalks",sidewalks,NaN,roads and sidewalks,Two people have already tripped and injured th...,0,two people already tripped injured night broke...,"[two, people, already, tripped, injured, night...","[two, peopl, alreadi, trip, injur, night, brok...","[two, people, already, tripped, injured, night..."
4,negative,street sign,street sign,Lyndale,traffic lights and signs,"New street sign says ""lyndal"" instead of ""lynd...",0,new street sign says lyndal instead lyndale,"[new, street, sign, says, lyndal, instead, lyn...","[new, street, sign, say, lyndal, instead, lyndal]","[new, street, sign, say, lyndal, instead, lynd..."


### Part of speech tagging

In [47]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/andreamock/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [48]:
def word_pos_tagger(text):
    pos_tagged_text = nltk.pos_tag(text)
    return pos_tagged_text

In [49]:
data_clean['text_tokens_pos_tagged'] = data_clean['text_tokens'].apply(lambda x: word_pos_tagger(x))
data_clean.head()

,sentiment,key words,objects,names,topic,review_text,label,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma,text_tokens_pos_tagged
0,negative,"sidewalk, asphalted, hazard, car",sidewalk,NaN,roads and sidewalks,The sidewalk/curb that extends from bike path ...,0,sidewalkcurb extends bike path road aligned dr...,"[sidewalkcurb, extends, bike, path, road, alig...","[sidewalkcurb, extend, bike, path, road, align...","[sidewalkcurb, extends, bike, path, road, alig...","[(sidewalkcurb, NN), (extends, VBZ), (bike, JJ..."
1,negative,"chairs, railing, fall","chairs, railing",NaN,municipal buildings,Chairs too close to railing. Kids could climb ...,0,chairs close railing kids could climb fall dea...,"[chairs, close, railing, kids, could, climb, f...","[chair, close, rail, kid, could, climb, fall, ...","[chair, close, railing, kid, could, climb, fal...","[(chairs, NNS), (close, RB), (railing, VBG), (..."
2,negative,"garbage, bin, color, trash","bin, city",NaN,cleanliness,THE COLOR OF A BIN SHOULD NOT MATTER. THURSDAY...,0,color bin matter thursday garbage day garbage ...,"[color, bin, matter, thursday, garbage, day, g...","[color, bin, matter, thursday, garbag, day, ga...","[color, bin, matter, thursday, garbage, day, g...","[(color, NN), (bin, NN), (matter, NN), (thursd..."
3,negative,"people, tripped, injured, broken, sidewalks",sidewalks,NaN,roads and sidewalks,Two people have already tripped and injured th...,0,two people already tripped injured night broke...,"[two, people, already, tripped, injured, night...","[two, peopl, alreadi, trip, injur, night, brok...","[two, people, already, tripped, injured, night...","[(two, CD), (people, NNS), (already, RB), (tri..."
4,negative,street sign,street sign,Lyndale,traffic lights and signs,"New street sign says ""lyndal"" instead of ""lynd...",0,new street sign says lyndal instead lyndale,"[new, street, sign, says, lyndal, instead, lyn...","[new, street, sign, say, lyndal, instead, lyndal]","[new, street, sign, say, lyndal, instead, lynd...","[(new, JJ), (street, NN), (sign, NN), (says, V..."


In [50]:
data_clean['text_cleaned'] = data_clean['text_tokens_lemma'].apply(lambda x: " ".join(x))
data_clean.head()

,sentiment,key words,objects,names,topic,review_text,label,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma,text_tokens_pos_tagged,text_cleaned
0,negative,"sidewalk, asphalted, hazard, car",sidewalk,NaN,roads and sidewalks,The sidewalk/curb that extends from bike path ...,0,sidewalkcurb extends bike path road aligned dr...,"[sidewalkcurb, extends, bike, path, road, alig...","[sidewalkcurb, extend, bike, path, road, align...","[sidewalkcurb, extends, bike, path, road, alig...","[(sidewalkcurb, NN), (extends, VBZ), (bike, JJ...",sidewalkcurb extends bike path road aligned dr...
1,negative,"chairs, railing, fall","chairs, railing",NaN,municipal buildings,Chairs too close to railing. Kids could climb ...,0,chairs close railing kids could climb fall dea...,"[chairs, close, railing, kids, could, climb, f...","[chair, close, rail, kid, could, climb, fall, ...","[chair, close, railing, kid, could, climb, fal...","[(chairs, NNS), (close, RB), (railing, VBG), (...",chair close railing kid could climb fall death...
2,negative,"garbage, bin, color, trash","bin, city",NaN,cleanliness,THE COLOR OF A BIN SHOULD NOT MATTER. THURSDAY...,0,color bin matter thursday garbage day garbage ...,"[color, bin, matter, thursday, garbage, day, g...","[color, bin, matter, thursday, garbag, day, ga...","[color, bin, matter, thursday, garbage, day, g...","[(color, NN), (bin, NN), (matter, NN), (thursd...",color bin matter thursday garbage day garbage ...
3,negative,"people, tripped, injured, broken, sidewalks",sidewalks,NaN,roads and sidewalks,Two people have already tripped and injured th...,0,two people already tripped injured night broke...,"[two, people, already, tripped, injured, night...","[two, peopl, alreadi, trip, injur, night, brok...","[two, people, already, tripped, injured, night...","[(two, CD), (people, NNS), (already, RB), (tri...",two people already tripped injured night broke...
4,negative,street sign,street sign,Lyndale,traffic lights and signs,"New street sign says ""lyndal"" instead of ""lynd...",0,new street sign says lyndal instead lyndale,"[new, street, sign, says, lyndal, instead, lyn...","[new, street, sign, say, lyndal, instead, lyndal]","[new, street, sign, say, lyndal, instead, lynd...","[(new, JJ), (street, NN), (sign, NN), (says, V...",new street sign say lyndal instead lyndale


## Finding the most common words

In [51]:
from collections import Counter

import string

In [52]:
def getMostCommonWords(reviews, n_most_common, stopwords=None):

    # flatten review column into a list of words, and set each to lowercase
    flattened_reviews = [word for review in reviews for word in \
                         review.lower().split()]


    # remove punctuation from reviews
    flattened_reviews = [''.join(char for char in review if \
                                 char not in string.punctuation) for \
                         review in flattened_reviews]


    # remove stopwords, if applicable
    if stopwords:
        flattened_reviews = [word for word in flattened_reviews if \
                             word not in stopwords]


    # remove any empty strings that were created by this process
    flattened_reviews = [review for review in flattened_reviews if review]

    return Counter(flattened_reviews).most_common(n_most_common)

In [53]:
getMostCommonWords(data_clean['text_cleaned'], 15)

[('park', 1128),
 ('walk', 559),
 ('place', 488),
 ('nice', 485),
 ('light', 389),
 ('beautiful', 370),
 ('street', 364),
 ('great', 346),
 ('field', 290),
 ('lot', 287),
 ('water', 266),
 ('sidewalk', 257),
 ('well', 253),
 ('soccer', 246),
 ('need', 233)]

## Text vectorization

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

In [55]:
vectorizer = TfidfVectorizer()
bow = vectorizer.fit_transform(list(data_clean['text_cleaned']))
labels = data_clean['label']

In [56]:
bow.shape

(3975, 4968)

In [57]:
len(vectorizer.get_feature_names())

4968

In [58]:
tfidfDict = dict(zip(vectorizer.get_feature_names(), bow.toarray()[0]))

In [59]:
tfidfDict['appreciated']

0.0

In [60]:
featureDf = pd.DataFrame.from_dict(tfidfDict, 
                                   orient='index', columns=['tfidf'])
featureDf.reset_index(inplace=True)
featureDf = featureDf.rename(columns = {'index':'feature'})

In [61]:
featureDf.sort_values('tfidf')[-10:]

,feature,tfidf
1305,driveway,0.220383
2925,onto,0.225119
1928,hazard,0.264554
522,bottom,0.267328
1504,extends,0.317696
111,aligned,0.317696
3910,sidewalkcurb,0.329263
1612,finishedasphalted,0.329263
3059,path,0.345577
648,car,0.369627


Now we can take a look at the words that have the highest tfidf score in the positive and negative sentiment datasets.

In [62]:
pos_reviews = data_clean[data_clean['label'] == 1]
neg_reviews = data_clean[data_clean['label'] == 0]

In [63]:
vectorizer_pos = TfidfVectorizer(min_df=15)
bow_pos = vectorizer_pos.fit_transform(list(pos_reviews['text_cleaned']))
labels_pos = pos_reviews['label']

In [64]:
vectorizer_neg = TfidfVectorizer(min_df=15)
bow_neg = vectorizer_neg.fit_transform(list(neg_reviews['text_cleaned']))
labels_neg = neg_reviews['label']

In [65]:
tfidfDictPos = dict(zip(vectorizer_pos.get_feature_names(), bow_pos.toarray()[0]))
tfidfDictNeg = dict(zip(vectorizer_neg.get_feature_names(), bow_neg.toarray()[0]))

In [66]:
posFeatureDf = pd.DataFrame.from_dict(tfidfDictPos, 
                                   orient='index', columns=['tfidf'])
posFeatureDf.reset_index(inplace=True)
posFeatureDf = posFeatureDf.rename(columns = {'index':'feature'})

In [67]:
negFeatureDf = pd.DataFrame.from_dict(tfidfDictNeg, 
                                   orient='index', columns=['tfidf'])
negFeatureDf.reset_index(inplace=True)
negFeatureDf = negFeatureDf.rename(columns = {'index':'feature'})

In [68]:
posFeatureDf.sort_values('tfidf')[-15:]

,feature,tfidf
53,friend,0.000000
54,fun,0.000000
55,game,0.000000
56,get,0.000000
57,go,0.000000
58,good,0.000000
59,grass,0.000000
61,green,0.000000
48,find,0.000000
47,field,0.000000


In [69]:
negFeatureDf.sort_values('tfidf')[-10:]

,feature,tfidf
110,fix,0.000000
109,first,0.000000
115,front,0.000000
286,street,0.184403
251,road,0.214221
89,driveway,0.293338
204,onto,0.302093
27,bike,0.349396
43,car,0.497851
216,path,0.610627


## Splitting data for train and test

In [70]:
neg_reviews.shape

(2000, 13)

In [71]:
pos_reviews.shape

(1975, 13)

In [72]:
vectorizerEn = TfidfVectorizer(min_df=15)
bowEn = vectorizerEn.fit_transform(data_clean['text_cleaned'])

In [73]:
X_train, X_test, y_train, y_test = train_test_split(bowEn,data_clean['label'], test_size=0.3)

In [74]:
# check out the dataset 
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2782, 504)
(1193, 504)
(2782,)
(1193,)


### Using Random Forest classifier 

In [91]:
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn import metrics

In [76]:
classifier = rfc()
classifier.fit(X_train,y_train)
classifier.score(X_test,y_test)

0.9488683989941324

In [77]:
print("train shape: " + str(X_train.shape))
print("score on test: " + str(classifier.score(X_test, y_test)))
print("score on train: "+ str(classifier.score(X_train, y_train)))

train shape: (2782, 504)
score on test: 0.9488683989941324
score on train: 0.9978432782171099


In [93]:
y_pred=classifier.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95       620
           1       0.93      0.96      0.95       573

    accuracy                           0.95      1193
   macro avg       0.95      0.95      0.95      1193
weighted avg       0.95      0.95      0.95      1193



## Decision Tree

In [78]:
%%time

from sklearn.tree import DecisionTreeClassifier

clfdt = DecisionTreeClassifier(min_samples_split=30,max_depth=10)
clfdt.fit(X_train, y_train)

print("train shape: " + str(X_train.shape))
print("score on test: "  + str(clfdt.score(X_test, y_test)))
print("score on train: " + str(clfdt.score(X_train, y_train)))

train shape: (2782, 504)
score on test: 0.8901927912824812
score on train: 0.9130122214234364
CPU times: user 26.4 ms, sys: 2.18 ms, total: 28.6 ms
Wall time: 27.8 ms


In [79]:
%%time

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bg=BaggingClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=3),max_samples=0.5,max_features=1.0,n_estimators=10)
bg.fit(X_train, y_train)

print("train shape: " + str(X_train.shape))
print("score on test: " + str(bg.score(X_test, y_test)))
print("score on train: "+ str(bg.score(X_train, y_train)))

train shape: (2782, 504)
score on test: 0.8189438390611903
score on train: 0.7936736161035226
CPU times: user 91.1 ms, sys: 4.11 ms, total: 95.2 ms
Wall time: 129 ms


In [80]:
# boosting decision tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# setting 
# min_samples_split=10
# max_depth=4

adb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=100,learning_rate=0.5)
adb.fit(X_train, y_train)

print("train shape: " + str(X_train.shape))
print("score on test: " + str(adb.score(X_test, y_test)))
print("score on train: "+ str(adb.score(X_train, y_train)))

train shape: (2782, 504)
score on test: 0.9471919530595139
score on train: 0.9978432782171099


In [81]:
X_train[5]

<1x504 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

## Naive Bayes

Sklearn Documentation:

- Naive Bayes: https://scikit-learn.org/stable/modules/naive_bayes.html
- MultinomialNB: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB

In [82]:
from sklearn.naive_bayes import MultinomialNB

In [83]:
%%time
mnb = MultinomialNB().fit(X_train, y_train)

CPU times: user 3.95 ms, sys: 1.85 ms, total: 5.8 ms
Wall time: 5.75 ms


In [84]:
print("train shape: " + str(X_train.shape))
print("score on test: " + str(mnb.score(X_test, y_test)))
print("score on train: "+ str(mnb.score(X_train, y_train)))

train shape: (2782, 504)
score on test: 0.9547359597652976
score on train: 0.9554277498202732


## Logistic Regression 

Sklearn Documentation:

- LogisticRegression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- SGD Classifier: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier

In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [86]:
%%time

lr=LogisticRegression(max_iter=5000)
lr.fit(X_train, y_train)

CPU times: user 151 ms, sys: 22.5 ms, total: 173 ms
Wall time: 269 ms


LogisticRegression(max_iter=5000)

In [87]:
print("train shape: " + str(X_train.shape))
print("score on test: " + str(lr.score(X_test, y_test)))
print("score on train: "+ str(lr.score(X_train, y_train)))

train shape: (2782, 504)
score on test: 0.961441743503772
score on train: 0.9726815240833933


In [88]:
%%time

#logistic regression with stochastic gradient decent
sgd=SGDClassifier()
sgd.fit(X_train, y_train)

CPU times: user 9.82 ms, sys: 2.67 ms, total: 12.5 ms
Wall time: 15.9 ms


SGDClassifier()

In [89]:
print("train shape: " + str(X_train.shape))
print("score on test: " + str(sgd.score(X_test, y_test)))
print("score on train: "+ str(sgd.score(X_train, y_train)))

train shape: (2782, 504)
score on test: 0.961441743503772
score on train: 0.9874191229331416


## K-nearest neighbors

In [90]:
%%time

from sklearn.neighbors import KNeighborsClassifier

#knn = KNeighborsClassifier(n_neighbors=5,algorithm = 'ball_tree')
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)

knn.fit(X_train, y_train)

print("train shape: " + str(X_train.shape))
print("score on test: " + str(knn.score(X_test, y_test)))
print("score on train: "+ str(knn.score(X_train, y_train)))

train shape: (2782, 504)
score on test: 0.8038558256496228
score on train: 0.8360891445003594
CPU times: user 680 ms, sys: 182 ms, total: 862 ms
Wall time: 742 ms


## Neural network pre-programmed

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#allReviewEn[['']]

train, test = train_test_split(allReviewEn, test_size = 0.3, random_state=42)

# clean the indexing
train.reset_index(drop=True),test.reset_index(drop=True)

# save train and test in csv files 
train[['review_text', 'label']].to_csv('all_en_train.csv', index=False)
test[['review_text', 'label']].to_csv('all_en_test.csv', index=False)

### Using Torchtest to processs text data

import numpy as np 

import torch 
import torchtext

from torchtext.legacy.data import Field, BucketIterator, TabularDataset, LabelField

import nltk 
nltk.download('punkt') # for punkt tokenizer

from nltk import word_tokenize 

# torchtext field parameter specifies how data should be processed, here tokenized
TEXT = Field(tokenize = word_tokenize)

LABEL = LabelField(dtype = torch.float) # convert 

datafields = [ ('review_text', TEXT), ('label', LABEL)] 

# specify what data that will work with, split to train and text, map to field 
trn, tst = TabularDataset.splits(path = '/Users/andreamock/Documents/review_datasets',
                               train = 'all_en_train.csv', test = 'all_en_test.csv', format = 'csv',
                               skip_header = True, fields = datafields)


# training examples 
trn[:5]

print(f'Number of training examples: {len(trn)}')
print(f'Number of testing examples: {len(tst)}')

# each example has label and text
trn[5].__dict__.keys()

trn[1].review_text # text has been tokenized in individual words

trn[1].label

# limit size of feature vectors to 15000, use one-encoding to get the top 15000 words in vocab
TEXT.build_vocab(trn, max_size = 15000)

LABEL.build_vocab(trn)

print(f'Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}')
print(f'Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}')
# two additional tokens were added to vocab, one for unknown words and another for padding to make sentences equal lengths

print(TEXT.vocab.freqs.most_common(50)) 

print(TEXT.vocab.itos[:10]) # integer to string mapping 0 and 1 to unknown and padding

batch_size = 64 

# returns a batch of examples where each example is of similar length (thus minimizing padding for each example)
train_iterator, test_iterator = BucketIterator.splits(
    (trn,tst), batch_size = batch_size, sort_key = lambda x: len(x.review_text), sort_within_batch = False
)

## Designing an RNN for binary text classification 

import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        # input_dim = input dimensions of words 
        # embedding_dim = dimension of word embeddings, dense word representation for training RNN
        # hidden_dim = dimension of hidden state of RNN
        # output_dim = output dimensions of RNN output
        
        super().__init__()
        #  convert one-hot encoded sentences to dense format using embeddings to represent each word
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        # input to rnn is current word's embedding and previous hidden state, one word per time instance (memory cell)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        # fully connected layer to classify as positive or negative 
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, text):
        # input sentence (list of indexes of one hot encoded words) is represented using its embedding
        embedded = self.embedding(text)
        
        embedded_dropout = self.dropout(embedded)
        
        # output = concatentation of hidden state for every time step (ie word) [sentence length, batch size, hiddendim]
        # hidden = final hidden state fed into linear layer
        output, (hidden, _) = self.rnn(embedded_dropout)
        
        hidden_1D = hidden.squeeze(0) # get rid of unnecessary dimension 
        
        assert torch.equal(output[-1, :, :], hidden_1D) # confirm that it is indeed last hidden state 
        
        return self.fc(hidden_1D) # last hidden state fed into fully connected layer

# setting dimensions 
input_dim = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim=1

model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)

model # see what our model looks like

# train with optimizer
import torch.optim as optim 

optimizer = optim.Adam(model.parameters(), lr=1e-6)

# binary cross entropy with logits (cross-entropy for binary classification, 
# w/ sigmoid activation func to predict in range of 0 and 1)
criterion = nn.BCEWithLogitsLoss()

def train(model, iterator, optimizer, criterion): # helper function for training process
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:  # iterator over all batches of training data
        
        optimizer.zero_grad() # zero out gradients of optimizer
                
        predictions = model(batch.review_text).squeeze(1) # make predictions, squeeze to be 1d instead of [, ]
        
        loss = criterion(predictions, batch.label) # calculate loss
        
        rounded_preds = torch.round(torch.sigmoid(predictions))
        correct = (rounded_preds == batch.label).float() # how many were correct
        
        acc = correct.sum() / len(correct)
        
        loss.backward() # backward pass on rnn
        
        optimizer.step()
        
        epoch_loss += loss.item() # keep track of epoch loss and accuracy
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

num_epochs = 5

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    print(f' Epoch: {epoch+1}, Train loss: {train_loss}, Train Acc: {train_acc*100:.2f}%')

Now we can test the accuracy on our test data.

# don't want to update the parameters when evaluating the accuracy
epoch_loss = 0
epoch_acc = 0

model.eval()

with torch.no_grad():

    for batch in test_iterator:

        predictions = model(batch.review_text).squeeze(1)

        loss = criterion(predictions, batch.label)

        rounded_preds = torch.round(torch.sigmoid(predictions))
        
        correct = (rounded_preds == batch.label).float() 
        acc = correct.sum() / len(correct)

        epoch_loss += loss.item()
        epoch_acc += acc.item()

test_loss = epoch_loss / len(test_iterator)
test_acc  = epoch_acc / len(test_iterator)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')